# Ventilator pressure prediction
we'll be using XGBoost regressor, catboost, lgbm libraries to predict our output. But I found XGBoost worked best for me without any finetuning. Other models will too work well if implemented properly.

[amitnikhade.com](https://amitnikhade.com)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ventilator-pressure-prediction/sample_submission.csv
/kaggle/input/ventilator-pressure-prediction/train.csv
/kaggle/input/ventilator-pressure-prediction/test.csv


In [2]:
# import tensorflow as tf
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# # instantiating the model in the strategy scope creates the model on the TPU


Load data and analyse it

In [3]:
df = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')

In [4]:
df

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987
...,...,...,...,...,...,...,...,...
6035995,6035996,125749,50,10,2.504603,1.489714,1,3.869032
6035996,6035997,125749,50,10,2.537961,1.488497,1,3.869032
6035997,6035998,125749,50,10,2.571408,1.558978,1,3.798729
6035998,6035999,125749,50,10,2.604744,1.272663,1,4.079938


In [5]:
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
test

,id,breath_id,R,C,time_step,u_in,u_out
0,1,0,5,20,0.000000,0.000000,0
1,2,0,5,20,0.031904,7.515046,0
2,3,0,5,20,0.063827,14.651675,0
3,4,0,5,20,0.095751,21.230610,0
4,5,0,5,20,0.127644,26.320956,0
...,...,...,...,...,...,...,...
4023995,4023996,125748,20,10,2.530117,4.971245,1
4023996,4023997,125748,20,10,2.563853,4.975709,1
4023997,4023998,125748,20,10,2.597475,4.979468,1
4023998,4023999,125748,20,10,2.631134,4.982648,1


In [6]:
df.describe()

,id,breath_id,R,C,time_step,u_in,u_out,pressure
count,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06
mean,3.018000e+06,6.283886e+04,2.703618e+01,2.608072e+01,1.307225e+00,7.321615e+00,6.204493e-01,1.122041e+01
std,1.742443e+06,3.633526e+04,1.959549e+01,1.715231e+01,7.659778e-01,1.343470e+01,4.852752e-01,8.109703e+00
min,1.000000e+00,1.000000e+00,5.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-1.895744e+00
25%,1.509001e+06,3.137700e+04,5.000000e+00,1.000000e+01,6.428995e-01,3.936623e-01,0.000000e+00,6.329607e+00
50%,3.018000e+06,6.276550e+04,2.000000e+01,2.000000e+01,1.308123e+00,4.386146e+00,1.000000e+00,7.032628e+00
75%,4.527000e+06,9.430100e+04,5.000000e+01,5.000000e+01,1.965502e+00,4.983895e+00,1.000000e+00,1.364103e+01
max,6.036000e+06,1.257490e+05,5.000000e+01,5.000000e+01,2.937238e+00,1.000000e+02,1.000000e+00,6.482099e+01


In [7]:
df.isna().any()

id           False
breath_id    False
R            False
C            False
time_step    False
u_in         False
u_out        False
pressure     False
dtype: bool

In [8]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6036000 entries, 0 to 6035999
Data columns (total 8 columns):
 #   Column     Dtype  
---  ------     -----  
 0   id         int64  
 1   breath_id  int64  
 2   R          int64  
 3   C          int64  
 4   time_step  float64
 5   u_in       float64
 6   u_out      int64  
 7   pressure   float64
dtypes: float64(3), int64(5)
memory usage: 368.4 MB


split data into features and label

In [9]:
X = df.drop('pressure', axis=1)

In [10]:
y = df['pressure']

In [11]:
# X = X[:5000]
# y = y[:5000]

In [12]:
# i.shape
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33)

In [13]:
# !pip install lightgbm --install-option=--gpu
# !pip install lightgbm --install-option=--cuda
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import Normalizer

## XGBoost

In [14]:
xgb = XGBRegressor(seed = 2021, n_estimators=500,verbosity=1, eval_metric="mae", tree_method="gpu_hist",gpu_id=0)
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eval_metric='mae', gamma=0,
             gpu_id=0, importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=2, num_parallel_tree=1, random_state=2021,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2021,
             subsample=1, tree_method='gpu_hist', validate_parameters=1,
             verbosity=1)

performance

In [15]:
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(xgb, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

Mean MAE: 1.966 (0.004)


In [16]:

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(xgb, X_valid, y_valid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

Mean MAE: 1.974 (0.008)


In [17]:
pred = xgb.predict(test)
sub = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')
sub['pressure'] = pred
sub.to_csv('submission.csv', index = 0)

## LGBM

In [18]:
lgbm = LGBMRegressor(random_state=2021,n_estimators=10, metric="mae", device_type="gpu",gpu_platform_id = 0, gpu_device_id = 0)
lgbm.fit(X_train, y_train)

LGBMRegressor(device_type='gpu', gpu_device_id=0, gpu_platform_id=0,
              metric='mae', n_estimators=10, random_state=2021)

performance

In [19]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(lgbm, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Mean MAE: 3.294 (0.005)


In [20]:

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(lgbm, X_valid, y_valid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

Mean MAE: 3.289 (0.010)


## CatBoost

In [21]:
cat = CatBoostRegressor(iterations=50, depth=3,eval_metric="MAE", learning_rate=0.1, task_type="GPU",devices="0")
cat.fit(X_train, y_train)

0:	learn: 5.7802063	total: 17.9ms	remaining: 876ms
1:	learn: 5.4162562	total: 30.6ms	remaining: 734ms
2:	learn: 5.1010885	total: 43ms	remaining: 674ms
3:	learn: 4.8279082	total: 56.1ms	remaining: 645ms
4:	learn: 4.5868560	total: 68.9ms	remaining: 620ms
5:	learn: 4.3801312	total: 82ms	remaining: 601ms
6:	learn: 4.2036829	total: 94.7ms	remaining: 582ms
7:	learn: 4.0542546	total: 108ms	remaining: 568ms
8:	learn: 3.9197502	total: 121ms	remaining: 553ms
9:	learn: 3.7940313	total: 134ms	remaining: 538ms
10:	learn: 3.6851493	total: 148ms	remaining: 523ms
11:	learn: 3.5887001	total: 161ms	remaining: 509ms
12:	learn: 3.5072646	total: 174ms	remaining: 495ms
13:	learn: 3.4365261	total: 187ms	remaining: 482ms
14:	learn: 3.3695059	total: 201ms	remaining: 468ms
15:	learn: 3.3104814	total: 214ms	remaining: 454ms
16:	learn: 3.2615929	total: 242ms	remaining: 470ms
17:	learn: 3.2138213	total: 260ms	remaining: 462ms
18:	learn: 3.1705934	total: 278ms	remaining: 453ms
19:	learn: 3.1399822	total: 295ms	rema

performance

In [22]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(cat, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

0:	learn: 5.7806099	total: 42.5ms	remaining: 2.08s
1:	learn: 5.4169197	total: 79.7ms	remaining: 1.91s
2:	learn: 5.1001580	total: 115ms	remaining: 1.8s
3:	learn: 4.8296830	total: 146ms	remaining: 1.68s
4:	learn: 4.5923541	total: 184ms	remaining: 1.65s
5:	learn: 4.3829601	total: 226ms	remaining: 1.66s
6:	learn: 4.2064633	total: 262ms	remaining: 1.61s
7:	learn: 4.0539565	total: 297ms	remaining: 1.56s
8:	learn: 3.9205068	total: 327ms	remaining: 1.49s
9:	learn: 3.7973252	total: 365ms	remaining: 1.46s
10:	learn: 3.6869864	total: 404ms	remaining: 1.43s
11:	learn: 3.5960662	total: 441ms	remaining: 1.4s
12:	learn: 3.5097596	total: 479ms	remaining: 1.36s
13:	learn: 3.4330999	total: 545ms	remaining: 1.4s
14:	learn: 3.3642578	total: 569ms	remaining: 1.33s
15:	learn: 3.3031059	total: 592ms	remaining: 1.26s
16:	learn: 3.2531057	total: 621ms	remaining: 1.21s
17:	learn: 3.2082249	total: 657ms	remaining: 1.17s
18:	learn: 3.1670615	total: 681ms	remaining: 1.11s
19:	learn: 3.1285589	total: 712ms	remainin

0:	learn: 5.7820413	total: 41.6ms	remaining: 2.04s
1:	learn: 5.4159520	total: 83.5ms	remaining: 2s
2:	learn: 5.1030495	total: 120ms	remaining: 1.88s
3:	learn: 4.8283132	total: 163ms	remaining: 1.87s
4:	learn: 4.5909971	total: 203ms	remaining: 1.82s
5:	learn: 4.3872338	total: 248ms	remaining: 1.82s
6:	learn: 4.2074903	total: 289ms	remaining: 1.78s
7:	learn: 4.0464875	total: 328ms	remaining: 1.72s
8:	learn: 3.9088938	total: 368ms	remaining: 1.68s
9:	learn: 3.7910146	total: 408ms	remaining: 1.63s
10:	learn: 3.6863089	total: 447ms	remaining: 1.58s
11:	learn: 3.5943727	total: 486ms	remaining: 1.54s
12:	learn: 3.5085990	total: 503ms	remaining: 1.43s
13:	learn: 3.4292932	total: 519ms	remaining: 1.33s
14:	learn: 3.3624217	total: 552ms	remaining: 1.29s
15:	learn: 3.3080033	total: 582ms	remaining: 1.24s
16:	learn: 3.2541951	total: 614ms	remaining: 1.19s
17:	learn: 3.2075216	total: 648ms	remaining: 1.15s
18:	learn: 3.1695378	total: 672ms	remaining: 1.1s
19:	learn: 3.1311872	total: 711ms	remaining

In [23]:

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(cat, X_valid, y_valid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

48:	learn: 2.7658996	total: 954ms	remaining: 19.5ms
49:	learn: 2.7614658	total: 979ms	remaining: 0us
0:	learn: 5.7799428	total: 30.3ms	remaining: 1.48s
1:	learn: 5.4156790	total: 65.1ms	remaining: 1.56s
2:	learn: 5.1000888	total: 99.9ms	remaining: 1.56s
3:	learn: 4.8272845	total: 124ms	remaining: 1.42s
4:	learn: 4.5861187	total: 150ms	remaining: 1.35s
5:	learn: 4.3812155	total: 176ms	remaining: 1.29s
6:	learn: 4.1969309	total: 200ms	remaining: 1.23s
7:	learn: 4.0402683	total: 224ms	remaining: 1.17s
8:	learn: 3.9061128	total: 248ms	remaining: 1.13s
9:	learn: 3.7844316	total: 272ms	remaining: 1.09s
10:	learn: 3.6793127	total: 295ms	remaining: 1.05s
11:	learn: 3.5874996	total: 320ms	remaining: 1.01s
12:	learn: 3.5018996	total: 344ms	remaining: 978ms
13:	learn: 3.4271285	total: 368ms	remaining: 945ms
14:	learn: 3.3627593	total: 392ms	remaining: 914ms
15:	learn: 3.3021514	total: 422ms	remaining: 898ms
16:	learn: 3.2533970	total: 447ms	remaining: 867ms
17:	learn: 3.2079549	total: 472ms	remai

## GRU RNN

In [24]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout, Flatten

def baseline_model():
        model = Sequential()
        model.add(GRU(32, kernel_initializer='normal', activation='relu',return_sequences=True, input_shape=(1, 7)))
        model.add(Dropout(0.2))
        model.add(GRU(32, kernel_initializer='normal', return_sequences=True,activation='relu'))
        model.add(Flatten())
        model.add(Dense(64, kernel_initializer='normal', activation='relu'))
        model.add(Dense(1,kernel_initializer='normal'))
        model.compile(loss='mean_absolute_error', optimizer='adam')
        return model

estimator = KerasRegressor(build_fn=baseline_model, batch_size=16, epochs=2, verbose=1) # increase the epoch to around 20 for better result
kfold = KFold(n_splits=2)  # replace the split with 5
results = cross_val_score(estimator, np.expand_dims(X, axis=1), y,scoring='neg_mean_absolute_error', cv=kfold)
print("Baseline: %.2f (%.2f) MAE" % (results.mean(), results.std()))

2021-09-26 21:56:21.390246: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-26 21:56:25.295151: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-26 21:56:25.297496: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-26 21:56:25.299856: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-26 21:56:25.300580: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-09-26 21:56:25.300675: I tensorflow/stream_executor/platform/def

Epoch 1/2


2021-09-26 21:56:29.149299: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11


    30/188625 [..............................] - ETA: 11:08 - loss: 537.5900

2021-09-26 21:56:30.034081: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


 76877/188625 [===========>..................] - ETA: 7:01 - loss: 6.680046:	learn: 2.7824888	total: 1.87s	remaining: 120ms
47:	learn: 2.7774810	total: 1.92s	remaining: 80ms
48:	learn: 2.7680572	total: 1.97s	remaining: 40.3ms
49:	learn: 2.7638256	total: 2s	remaining: 0us
0:	learn: 5.7703303	total: 24.6ms	remaining: 1.2s
1:	learn: 5.4061495	total: 51.3ms	remaining: 1.23s
2:	learn: 5.0896077	total: 75.5ms	remaining: 1.18s
3:	learn: 4.8170082	total: 98.7ms	remaining: 1.14s
4:	learn: 4.5810463	total: 121ms	remaining: 1.09s
5:	learn: 4.3726231	total: 145ms	remaining: 1.06s
6:	learn: 4.1915957	total: 168ms	remaining: 1.03s
7:	learn: 4.0379708	total: 193ms	remaining: 1.01s
8:	learn: 3.9050908	total: 215ms	remaining: 980ms
9:	learn: 3.7826899	total: 238ms	remaining: 954ms
10:	learn: 3.6748822	total: 263ms	remaining: 933ms
11:	learn: 3.5856882	total: 288ms	remaining: 913ms
12:	learn: 3.5043892	total: 310ms	remaining: 883ms
13:	learn: 3.4289739	total: 331ms	remaining: 852ms
14:	learn: 3.3602883	

 76936/188625 [===========>..................] - ETA: 7:02 - loss: 6.678945:	learn: 2.7871241	total: 1.48s	remaining: 129ms
46:	learn: 2.7825909	total: 1.52s	remaining: 96.9ms
47:	learn: 2.7773106	total: 1.56s	remaining: 65.1ms
48:	learn: 2.7716362	total: 1.58s	remaining: 32.3ms
49:	learn: 2.7648550	total: 1.6s	remaining: 0us
0:	learn: 5.7715553	total: 23.2ms	remaining: 1.14s
1:	learn: 5.4065869	total: 41.3ms	remaining: 990ms
2:	learn: 5.0894855	total: 64.2ms	remaining: 1s
3:	learn: 4.8154742	total: 88.1ms	remaining: 1.01s
4:	learn: 4.5786981	total: 115ms	remaining: 1.03s
5:	learn: 4.3696764	total: 139ms	remaining: 1.02s
6:	learn: 4.1949468	total: 165ms	remaining: 1.01s
7:	learn: 4.0353429	total: 192ms	remaining: 1.01s
8:	learn: 3.9019862	total: 217ms	remaining: 990ms
9:	learn: 3.7803912	total: 243ms	remaining: 974ms
10:	learn: 3.6761175	total: 268ms	remaining: 950ms
11:	learn: 3.5794886	total: 295ms	remaining: 936ms
12:	learn: 3.4932183	total: 324ms	remaining: 922ms
13:	learn: 3.41993

188625/188625 [==============================] - 738s 4ms/step - loss: 5.8024
Epoch 2/2
188625/188625 [==============================] - 312s 2ms/step
Epoch 1/2
188625/188625 [==============================] - 751s 4ms/step - loss: 5.5204
Epoch 2/2
188625/188625 [==============================] - 317s 2ms/step
Baseline: -5.08 (0.03) MAE


In [25]:
# train with the same configuration as below
# from tensorflow.keras.callbacks import EarlyStopping
# early_stopping = EarlyStopping(monitor='val_loss', patience = 3, restore_best_weights=True )
# history = estimator.fit(np.expand_dims(X, axis=1),y,validation_split=0.2, epochs=20,callbacks=[early_stopping])

## GRU TPU

Train with TPU

In [26]:
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout, Flatten

# def baseline_model():
#     with tpu_strategy.scope():
#         model = Sequential()
#         model.add(GRU(32, kernel_initializer='normal', activation='relu',return_sequences=True, input_shape=(1, 7)))
#         model.add(Dropout(0.2))
#         model.add(GRU(32, kernel_initializer='normal', return_sequences=True,activation='relu'))
#         model.add(Flatten())
#         model.add(Dense(64, kernel_initializer='normal', activation='relu'))
#         model.add(Dense(1,kernel_initializer='normal'))
#         model.compile(loss='mean_absolute_error', optimizer='adam')
#         return model

# estimator = KerasRegressor(build_fn=baseline_model, batch_size=16, verbose=1)
# kfold = KFold(n_splits=5)
# results = cross_val_score(estimator, np.expand_dims(X_train, axis=1), y_train,scoring='neg_mean_absolute_error', cv=kfold)
# print("Baseline: %.2f (%.2f) MAE" % (results.mean(), results.std()))

In [27]:
# from tensorflow.keras.callbacks import EarlyStopping
# early_stopping = EarlyStopping(monitor='val_loss', patience = 3, restore_best_weights=True )
# history = estimator.fit(np.expand_dims(X, axis=1),y,validation_split=0.2, epochs=20,callbacks=[early_stopping])

In [28]:
# history.history['loss']

In [29]:
# history.history['val_loss']

In [30]:
# import matplotlib.pyplot as plt
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [31]:
# Thanks!